Cross-validation for a time series data:
for k in range(2, 10):
    Train on 1-k^th day, test on (k+1)^th.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

## Develop cross validation

In [15]:
df = pd.read_csv('data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")

## Model 1 (baseline)

In [2]:
# create a map: colName -> index
def build_colname_idx(df):
    return dict(zip(df.columns, range(len(df.columns))))

model1_cols = ['click',
               'C1',
             'banner_pos',
             'site_category',
             'app_category',
             'device_type',
             'device_conn_type',
             'C15',
             'C16',
             'C18',
             'C19',
             'C21']

str_cols = ['site_category', 'app_category']

In [3]:
df = pd.read_csv('data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")

In [4]:
def fit_encoders(df):
    site_category_encoder = LabelEncoder()
    site_category_encoder.fit(df.site_category)
    app_category_encoder = LabelEncoder()
    app_category_encoder.fit(df.app_category)
    return site_category_encoder, app_category_encoder

In [5]:
# LabelEncoder doesn't handle unknown values, so fit them to the entire dataset.
site_category_encoder, app_category_encoder = fit_encoders(df)

In [6]:
def train_dev_test_split(df):
    last_day = 30
    last_day_mask = df.hour.dt.day == last_day
    df_test = df[last_day_mask]
    df_test = df_test[model1_cols]
    df_dev, df_test = train_test_split(df_test, test_size=0.5, random_state=23)
    df_train = df[~last_day_mask]
    df_train = df_train[model1_cols]
    X_train = df_train.drop('click', axis=1)
    y_train = df_train.click
    X_dev = df_dev.drop('click', axis=1)
    y_dev = df_dev.click
    X_test = df_test.drop('click', axis=1)
    y_test = df_test.click
    return X_train, y_train, X_dev, y_dev, X_test, y_test

In [7]:
# train/dev/test split
X_train, y_train, X_dev, y_dev, X_test, y_test = train_dev_test_split(df)

In [8]:
def fit_transform_train(X_train, site_category_encoder, app_category_encoder):
    X_train.site_category = site_category_encoder.transform(X_train.site_category)
    X_train.app_category = app_category_encoder.transform(X_train.app_category)
    X_train = X_train.values
    # when transforming, an unknown categorical feature is mapped to a zero vector
    oh_encoder = OneHotEncoder(handle_unknown='ignore')
    X_train = oh_encoder.fit_transform(X_train)
    return X_train, oh_encoder

In [9]:
train_colname_idx = build_colname_idx(X_train)
train_colname_idx

{'C1': 0,
 'banner_pos': 1,
 'site_category': 2,
 'app_category': 3,
 'device_type': 4,
 'device_conn_type': 5,
 'C15': 6,
 'C16': 7,
 'C18': 8,
 'C19': 9,
 'C21': 10}

In [10]:
# prepare the train set
X_train, oh_encoder = fit_transform_train(X_train, site_category_encoder, app_category_encoder)

In [11]:
# tune: C. Maybe use class_weight. Try different solvers.
# Can be parallelized if multi-class.
lg = LogisticRegression()
lg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
def transform_dev(X_dev, site_category_encoder, app_category_encoder, oh_encoder):
    X_dev.site_category = site_category_encoder.transform(X_dev.site_category)
    X_dev.app_category = app_category_encoder.transform(X_dev.app_category)
    X_dev = oh_encoder.transform(X_dev)
    return X_dev

In [13]:
# prepare the dev set
X_dev = transform_dev(X_dev, site_category_encoder, app_category_encoder, oh_encoder)

In [14]:
lg.score(X_dev, y_dev)

0.7547169811320755